In [11]:
import torch
from model import ResNet50
from BoTNet import botnet
import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from preprocess import DistortedKadis700k, DistortedKadid10k
from torchvision import transforms as T
from torch.utils.data import DataLoader
import os


In [ ]:
device = "cuda:0"
model = models.resnet50(weights="DEFAULT")
# model.classifier[-1] = nn.Linear(in_features=4096, out_features=125, bias=True)
model


In [12]:
dpm = botnet(
    '/home/sharfikeg/my_files/VIPNet/pretrained_model/botnet_model_best.pth.tar',
    resolution=(288, 384), heads=16, num_classes=150
)

=> Using pre-trained distortion perception module from /home/sharfikeg/my_files/VIPNet/pretrained_model/botnet_model_best.pth.tar.


In [15]:
dpm.fc[1]

Linear(in_features=8192, out_features=150, bias=True)

In [14]:
x = torch.randn([2, 3, 288, 384], device=device)
output = model(x)
F.softmax(output, dim=-1).max(1)

torch.return_types.max(
values=tensor([0.0135, 0.0154], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([29, 90], device='cuda:0'))

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

In [19]:
transform = T.Compose([
    T.Resize((288, 384)),
    T.ToTensor()
])

dataset = DistortedKadis700k('/extra_disk_1/s-kastryulin/data/kadis700k/images/',  transform=transform)

loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=4)

In [ ]:
path = "/home/sharfikeg/my_files/retIQA/csiq/distorted_images/"
for dir in os.listdir(path):
    cur_path = path + dir
    for name in os.listdir(cur_path):
        ref_name, dst_type, level, ext = name.split('.')
        if dst_type == 'AWGN':
            new_dst_type = '0'
        elif dst_type == 'BLUR':
            new_dst_type = '1'
        elif dst_type == 'contrast':
            new_dst_type = '2'
        elif dst_type == 'fnoise':
            new_dst_type = '3'
        elif dst_type == 'JPEG':
            new_dst_type = '4'
        elif dst_type == 'jpeg2000':
            new_dst_type = '5'
        new_name = f"{ref_name}_{new_dst_type}_{level}.{ext}"
        print(name, new_name)
        